In [1]:
import minsearch

In [2]:
import json

In [3]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [5]:
documents[0]

{'text': "Anaconda3-2024.02-1-Windows-x86_64.exe\nThe purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"], keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [7]:
q = "the course has already started, can I still enroll?"

In [8]:
index.fit(documents)

In [9]:
from ollama import Client

# import asyncio
# from ollama import AsyncClient

In [10]:
client = Client()
response = client.chat(
    model="llama3:instruct", messages=[{"role": "user", "content": q}]
)

print(response["message"]["content"])

I think there may be some confusion here!

As a conversational AI, I don't have the ability to enroll you in a course or provide information about a specific course. Additionally, we didn't start any conversation about a course before this.

So, to answer your question, it's not possible for me to help you enroll in a course that has already started. But if you're interested in learning more about a particular topic or subject, I'd be happy to help you explore resources and information related to that area!


In [11]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}

    results = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5,
    )

    return results

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = (
            context
            + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        )

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [13]:
def llm(prompt: str, client: Client):
    response = client.chat(
        model="llama3:instruct",
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
    )
    return response["message"]["content"]

In [14]:
query = "how do I run kafka?"


def rag(query, client: Client):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, client)
    return answer

In [15]:
rag(query, client)

'To run Kafka, you can follow the steps provided in the "Java Kafka: How to run producer/consumer/kstreams/etc in terminal" section:\n\nIn the project directory, run:\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nThis assumes that you have already built and packaged your Kafka application into a JAR file, and that the `build/libs` folder contains the compiled JAR.'

In [16]:
print(rag("the course has already started, can I still enroll?", client))

Based on the CONTEXT from the FAQ database, I'd answer the QUESTION: "the course has already started, can I still enroll?" as follows:

Yes, even if the course has already started, you're still eligible to submit homeworks. However, be aware that there will be deadlines for turning in final projects, so don't leave everything for the last minute.


In [17]:
documents[0]

{'text': "Anaconda3-2024.02-1-Windows-x86_64.exe\nThe purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [18]:
from elasticsearch import Elasticsearch

In [19]:
es_client = Elasticsearch("http://localhost:9200")

In [22]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [23]:
documents[0]

{'text': "Anaconda3-2024.02-1-Windows-x86_64.exe\nThe purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [24]:
from tqdm.auto import tqdm

In [25]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/1002 [00:00<?, ?it/s]

In [26]:
query = "I just disovered the course. Can I still join it?"

In [27]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields",
                    }
                },
                "filter": {"term": {"course": "data-engineering-zoomcamp"}},
            }
        },
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs

In [28]:
def rag(query, client):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, client)
    return answer

In [29]:
print(rag(query, client))

Based on the context, I would answer your question as follows:

Since you've just discovered the course, yes, you can still join it! According to the FAQ, even if you don't register, you're still eligible to submit homeworks. Just be aware that there will be deadlines for turning in the final projects, so don't leave everything for the last minute.

You can start by installing and setting up all the dependencies and requirements, looking over the prerequisites and syllabus, and getting comfortable with the subjects before the course starts.
